<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D1_MongoDB_Local_CRUD_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Executed on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Executed on  2024-09-03 08:12:27.990274+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


Based on https://github.com/giffy/MongoDB_PyMongo_Tutorial

#Installs

In [2]:
#check which version of MongoDB / pymongo is available
#!apt-cache policy mongodb
#!pip show pymongo
!pip3 index versions pymongo

pymongo (4.8.0)
Available versions: 4.8.0, 4.7.3, 4.7.2, 4.7.1, 4.7.0, 4.6.3, 4.6.2, 4.6.1, 4.6.0, 4.5.0, 4.4.1, 4.4.0, 4.3.3, 4.3.2, 4.2.0, 4.1.1, 4.1.0, 4.0.2, 4.0.1, 4.0, 3.13.0, 3.12.3, 3.12.2, 3.12.1, 3.12.0, 3.11.4, 3.11.3, 3.11.2, 3.11.1, 3.11.0, 3.10.1, 3.10.0, 3.9.0, 3.8.0, 3.7.2, 3.7.1, 3.7.0, 3.6.1, 3.6.0, 3.5.1, 3.5.0, 3.4.0, 3.3.1, 3.3.0, 3.2.2, 3.2.1, 3.2, 3.1.1, 3.1, 3.0.3, 3.0.2, 3.0.1, 3.0, 2.9.5, 2.9.4, 2.9.3, 2.9.2, 2.9.1, 2.9, 2.8.1, 2.8, 2.7.2, 2.7.1, 2.7, 2.6.3, 2.6.2, 2.6.1, 2.6, 2.5.2, 2.5.1, 2.5, 2.4.2, 2.4.1, 2.4, 2.3, 2.2.1, 2.2, 2.1.1, 2.1, 2.0.1, 2.0, 1.11, 1.10.1, 1.10, 1.9, 1.8.1, 1.8, 1.7, 1.6, 1.5.2, 1.5.1, 1.5, 1.4, 1.3, 1.2.1, 1.2, 1.1.2, 1.1.1, 1.1, 1.0, 0.16, 0.15.2, 0.15.1, 0.15, 0.14.2, 0.14.1, 0.14, 0.13, 0.12, 0.11.3, 0.11.2, 0.11.1, 0.11, 0.10.3, 0.10.2, 0.10.1, 0.10, 0.9.7, 0.9.6, 0.9.5, 0.9.4, 0.9.3, 0.9.2, 0.9.1, 0.9, 0.8.1, 0.8, 0.7.2, 0.7.1, 0.7, 0.6


In [3]:
#!curl https://pypi.org/pypi/pymongo/json

In [4]:
#!apt install mongodb > /dev/null

#Local Server Need Not be started for Remote Operations
#!service mongodb start
#!mongod --version

In [5]:
# Updated Install Sequence for MongoDB
# https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/
# Note Ubuntu versio 'jammy'
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg \
   --dearmor
# -----------------------
# Note reference to jammy
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
# -----------------------
!sudo apt-get update > /dev/null
!sudo apt-get install -y mongodb-org > /dev/null              # ignore error and warning messages

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [6]:
#!sudo service mongodb start --- this should work but does not
# -----------------------
# https://unix.stackexchange.com/questions/715119/system-v-init-service-cant-recongnise-mongod
!sudo mkdir -p /var/log/mongodb /var/lib/mongodb
!sudo chown `whoami` /var/lib/mongodb /var/log/mongodb/
# -----------------------
# Manually start MongoDB in background mode, to free up the terminal
!nohup mongod --config /etc/mongod.conf &
!mongod --version

nohup: appending output to 'nohup.out'
db version v6.0.17
Build Info: {
    "version": "6.0.17",
    "gitVersion": "1b0ca02043c6d35d5cfdc91e21fc00a05d901539",
    "openSSLVersion": "OpenSSL 3.0.2 15 Mar 2022",
    "modules": [],
    "allocator": "tcmalloc",
    "environment": {
        "distmod": "ubuntu2204",
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}


In [7]:
!pip -qq install pymongo
from pymongo import MongoClient
import pandas as pd
import dateutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.2 MB/s eta 0:00:00


In [8]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'config', 'local']

['admin', 'config', 'local']

# Create Database objects

In [9]:
# Database PraxisDB will be created only when we create collections, or documents in it
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names()

['admin', 'config', 'local']

In [10]:
# Delete a database
client.drop_database('PraxisDB')

In [11]:
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db = client.PraxisDB
db.create_collection('addresses')       # Optional collection creation

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PraxisDB'), 'addresses')

In [12]:
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'name': 'addresses',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xcf\xab\xe95\x06\xc0A\r\xaay\xb5]\xd3k#\x82', 4)},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [13]:
# Set the collection to work with
kollection = db.addresses
kollection.insert_one({'name' : 'jordi'})     # Insert one item to create the collection
list (kollection.find())                  # Show the existing collections

[{'_id': ObjectId('66d677c6555647d47d9f4b6d'), 'name': 'jordi'}]

In [14]:
# Rename a collections
db.addresses.rename('addressbook')
kollection2 = db.addressbook                   # Set the collection to work with
list (db.list_collections())                  # Show the existing collections

[{'name': 'addressbook',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xcf\xab\xe95\x06\xc0A\r\xaay\xb5]\xd3k#\x82', 4)},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [15]:
# Delete collection
db.drop_collection('addressbook')

{'nIndexesWas': 1, 'ns': 'PraxisDB.addressbook', 'ok': 1.0}

In [16]:
client.list_database_names()

['admin', 'config', 'local']

# Operations

## Single Row Insert

In [17]:
cDoc1 = {  'name' : 'Prithwis' ,                                     # String
          'age' : '59',                                             # Integer
          'gender' : 'Male',                                        # String
          'likes_code' : True,                                      # Boolean
          'address': {
              'street' : 'Panditiya Road',                          # String ( special character with escape \ )
              'number' : 36,                                        # Integer
              'city' : 'Kolkata',                                   # String
              'floor' : None,                                       # Null
              'postalcode' : '700029',                              # String containing a number
              },
          'favouritefruits': ['apple','pineapple','orange']         # Array
       }

In [18]:
# Define database, collection etc
db = client.PraxisDB
# Delete collection
db.drop_collection('AddressBook')
db.create_collection('AddressBook')       # Optional collection creation
kollection3 = db.AddressBook                   # Set the collection to work with
insert_result = kollection3.insert_one( cDoc1)
insert_result.acknowledged    # Confirms that insert is successful

True

In [19]:
insert_result.inserted_id     # Shows the document ID

ObjectId('66d677c6555647d47d9f4b6e')

In [20]:
db.AddressBook.find()  # returns a cursor

In [21]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('66d677c6555647d47d9f4b6e'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}


In [22]:
cDoc2 = {  'name' : 'Charanpreet' ,                                     # String
          'age' : '57',                                             # Integer
          'gender' : 'Male',                                        # String
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Kolkata',                                   # String
              'floor' : None,                                       # Null
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array
       }

In [23]:
insert_result = kollection3.insert_one( cDoc2)
insert_result.acknowledged    # Confirms that insert is successful

True

In [24]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('66d677c6555647d47d9f4b6e'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('66d677c6555647d47d9f4b6f'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}


In [25]:
cDoc3 = {  'name' : 'Narendra' ,                                     # String
          'age' : '67',                                             # Integer
          'gender' : 'Male',                                        # String
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Delhi',                                   # String
              'floor' : None,                                       # Null
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array
       }

cDoc4 = {  'name' : 'Aditya' ,                                     # String
          'age' : '47',                                             # Integer
          'gender' : 'Male',                                        # String
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Lucknow',                                   # String
              'floor' : None,                                       # Null
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array
       }

In [26]:
insert_result = kollection3.insert_many([cDoc3,cDoc4])

In [27]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('66d677c6555647d47d9f4b6e'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('66d677c6555647d47d9f4b6f'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('66d677c6555647d47d9f4b70'), 'name': 'Narendra', 'age': '67', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Delhi', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('66d677c6555647d47d9f4b71'), 'name': 'Aditya', 'age': '47', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Lucknow',

## Read

In [28]:
list(kollection3.find())

[{'_id': ObjectId('66d677c6555647d47d9f4b6e'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']},
 {'_id': ObjectId('66d677c6555647d47d9f4b6f'),
  'name': 'Charanpreet',
  'age': '57',
  'gender': 'Male',
  'likes_code': False,
  'address': {'street': None,
   'number': None,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700001'},
  'favouritefruits': ['apple', 'guava', 'mango']},
 {'_id': ObjectId('66d677c6555647d47d9f4b70'),
  'name': 'Narendra',
  'age': '67',
  'gender': 'Male',
  'likes_code': False,
  'address': {'street': None,
   'number': None,
   'city': 'Delhi',
   'floor': None,
   'postalcode': '700001'},
  'favouritefruits': ['apple', 'guava', 'mango']},
 {'_id': ObjectId('66d677c6555647d47d9f4b71'),
  'name': 'Aditya',
  'age': '47',
  'gender': 'Male'

In [29]:
#list ( collection.find( {'name' : 'Charanpreet' } ))
list ( kollection3.find( {'name' : 'Prithwis' } ))

[{'_id': ObjectId('66d677c6555647d47d9f4b6e'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

In [30]:
list ( kollection3.find( {'address.postalcode' : '700029' } ))             # find, can use one key or more

[{'_id': ObjectId('66d677c6555647d47d9f4b6e'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

## Update

In [31]:
## Update an existing document
update_result = kollection3.update_one(
    {'name' : 'Prithwis'},
    {'$set' : { 'age' : 101 }} )

list (kollection3.find( {'name' : 'Prithwis' } ))

# Alternative : collection.find_one_and_update( {'name' : Name}, {'$set' : { 'age' : 30 }} )

[{'_id': ObjectId('66d677c6555647d47d9f4b6e'),
  'name': 'Prithwis',
  'age': 101,
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

In [32]:
## Update an existing document
update_result = kollection3.update_one(
    {'name' : 'Prithwis'},
    {'$set' : { 'name' : 'Prithwis Mukerjee' }} )

list (kollection3.find( {'name' : 'Prithwis Mukerjee' } ))

[{'_id': ObjectId('66d677c6555647d47d9f4b6e'),
  'name': 'Prithwis Mukerjee',
  'age': 101,
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

## Additional stuff
https://www.analyticsvidhya.com/blog/2020/02/mongodb-in-python-tutorial-for-beginners-using-pymongo/

In [33]:
print('Executed on ',datetime.now(pytz.timezone('Asia/Calcutta')))

Executed on  2024-09-03 08:13:18.941611+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)